Testing running pytorch models using the common pipeline functions

In [1]:
import os, sys, json
import pandas as pd # for manipulating dataframes
import time
from datetime import date
import argparse
import torch


batch_utils_path = "/grand/gpu_hack/imlab/users/saideep/shared_folder/enformer_pipeline/scripts/modules"
sys.path.append(batch_utils_path)

import predictionUtils
import predictUtils_one
import predictUtils_two
import sequencesUtils
import enformer_pytorch
import attention
import loggerUtils

2023-05-04 16:42:26.882972: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using this config file: /grand/gpu_hack/imlab/users/saideep/test_inference/config.json


In [2]:
!nvidia-smi

Thu May  4 16:42:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.04   Driver Version: 470.103.04   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    52W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:46:00.0 Off |                    0 |
| N/A   

We can now start by building a test sequence

In [3]:
fasta_file = "/grand/TFXcan/imlab/data/hg_sequences/hg38/Homo_sapiens_assembly38.fasta"

test_region = "chr5_2000000_2000100"
seq_length_basenji = 131072
fasta_extractor = fasta_extractor = sequencesUtils.get_fastaExtractor(fasta_file)

test_seq = sequencesUtils.extract_reference_sequence(test_region, fasta_extractor, resize_for_enformer=False, resize_length=seq_length_basenji)

In [4]:
print(test_seq)
print(test_seq["sequence"].shape)

{'sequence': array([[[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]]], dtype=float32), 'interval_object': {'chr': 'chr5', 'start': 1934515, 'end': 2065587}}
(1, 131072, 4)


Great, we should load a test pytorch model now

In [5]:

model = enformer_pytorch.Enformer().to("cuda")
print(torch.cuda.is_available())
model.load_state_dict(torch.load("/grand/gpu_hack/imlab/users/temi/projects/alcf-hackathon-2023/outputs/outputs-2023-05-04-060420/model_1500.pt"))
model.eval()

True


Enformer(
  (_trunk): Sequential(
    (0): Sequential(
      (0): Rearrange('b l c -> b c l')
      (1): Conv1d(4, 768, kernel_size=(15,), stride=(1,), padding=same)
      (2): Residual(
        (_module): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,), padding=same)
        )
      )
      (3): SoftmaxPooling1D(
        (_logit_linear): Linear(in_features=768, out_features=768, bias=False)
      )
    )
    (1): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=same)
        )
        (1): Residual(
          (_module): Sequential(
            (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): GE

The model has been loaded, we can run the test inference:

In [6]:
test_out = model(torch.tensor(test_seq["sequence"]).to("cuda"))


This produces a dictionary of tensors. We want to extract just the human output and then remove the object from memory

In [7]:

human_out_cpu = test_out['human'].cpu().detach().numpy()

del test_out
torch.cuda.empty_cache()



This creates a copy of the human output into cpu (host) memory, and then we clear gpu space by deleting the original outputs.
Now we can save the outputs as an hdf5 file.

In [8]:

test_out = model(torch.tensor(test_seq["sequence"]).to("cuda"))

test_out_dir = "/grand/gpu_hack/imlab/users/saideep/test_inference"

sample_predictions = {
    "haplo1": human_out_cpu
}

test_meta = {
    "region" : test_region,
    "sequence_source" : "ref"
}

loggerUtils.save_haplotypes_h5_prediction(haplotype_predictions=sample_predictions, 
                                          metadata=test_meta, 
                                          output_dir=test_out_dir, sample="test")

['chr5_2000000_2000100', 'test', 'completed', 'ref']

In [ ]:
test_out_path = "/grand/gpu_hack/imlab/users/saideep/test_inference/test/haplo1/chr5_2000000_2000100_predictions.h5"


In [10]:

# FULL LOOP

fasta_file = "/grand/TFXcan/imlab/data/hg_sequences/hg38/Homo_sapiens_assembly38.fasta"

seq_length_basenji = 131072
fasta_extractor = fasta_extractor = sequencesUtils.get_fastaExtractor(fasta_file)

model = enformer_pytorch.Enformer().to("cuda")
print(torch.cuda.is_available())
model.load_state_dict(torch.load("/grand/gpu_hack/imlab/users/temi/projects/alcf-hackathon-2023/outputs/outputs-2023-05-04-060420/model_1500.pt"))
model.eval()

out_dir = "/grand/gpu_hack/imlab/users/saideep/test_inference"

with open("/grand/gpu_hack/imlab/data/basenji_data_pt/human/sequences.bed", "r") as seqs:
    for line in seqs:
        if line.strip().split("\t")[-1] != "test":
            continue
        cur_region = "_".join(line.strip().split("\t")[0:3])
        print(test_region)

        cur_seq = sequencesUtils.extract_reference_sequence(cur_region, fasta_extractor, resize_for_enformer=False, resize_length=seq_length_basenji)

        cur_out = model(torch.tensor(cur_seq["sequence"]).to("cuda"))
        
        human_out_cpu = cur_out['human'].cpu().detach().numpy()

        del cur_out
        torch.cuda.empty_cache()
        

        sample_predictions = {
            "ref": human_out_cpu
        }

        test_meta = {
            "region" : cur_region,
            "sequence_source" : "ref"
        }

        loggerUtils.save_haplotypes_h5_prediction(haplotype_predictions=sample_predictions, 
                                                metadata=test_meta, 
                                                output_dir=out_dir, sample="test")




True
chr5_2000000_2000100


NameError: name 'test_out' is not defined

42505273344 19547553792 9014983168


42505273344 19547553792 9015886848
